In [1]:
import os
import sys
import configparser

import pandas as pd

#add dark to path
src_code_dir = os.sep.join(os.path.abspath('').split(os.sep)[:-2])+os.sep
sys.path.insert(0,src_code_dir)

from dark.gateway import DarkGateway
from dark import DarkMap
from dark import DarkPid
from dark.decoder import DarkDecoder

load blockchain drivers, parameters and configurations

In [2]:
PROJECT_ROOT = 'D:\\workspace\\dark\\dark-env\\dARK\\'
config_file = os.path.join(PROJECT_ROOT,'config.ini')
#blockchain config
bc_config = configparser.ConfigParser()
bc_config.read(config_file)
#deployed contracts config
deployed_contracts_config = configparser.ConfigParser()
a = deployed_contracts_config.read(os.path.join(PROJECT_ROOT,'deployed_contracts.ini'))



path to data

In [3]:
DATA_PATH = 'D:\\workspace\\dark\\data\\'
df=pd.read_json(DATA_PATH+'pb-data.json')

configure the blockchain access

In [4]:
# Load Blockchain Drivers to the dARK GateWay
dgw =  DarkGateway(bc_config,deployed_contracts_config)
# create a map to interact with the Blockchain Smart Contracts
dm = DarkMap(dgw)

In [10]:
def get_pids(input_df,dark_map):

    oasis_dark_map = {}

    for index, row in input_df.iterrows():

        oasis_id = row['id']
        oai_id   = row['oai_identifier_str']
        title    = row.get('title')
        tipo     = row.get('format')
        url      = row.get('url')
        # print(index)
        # request, resp_time = measure_response_time(cc.request_pid)
        # request, resp_time, amount = measure_tx_params(dark_gw,cc.request_pid)
        try:
            ark = dark_map.sync_request_pid()
            status = 'ok'
        except KeyError:
            ark = '-'
            status = 'error'
    
    
        # oasis_dark_map[oasis_id] = {'dark' : ark, 'status' :  status , 'exec_time' : resp_time , 'amount' : amount , 'action' : 'assign_pid'}
        oasis_dark_map[oasis_id] = {'dark' : ark, 'status' :  status , 'action' : 'assign_pid'}

    return oasis_dark_map

In [12]:
def call_pid_method(cmd,input_df,pid_map,dark_map):

    # dark_gw = getDarkGateway()
    # cc = HyperDriveClient()
    # cr_client = CrossrefClient()

    action_map = {}
    
    for index, row in input_df.iterrows():

        oasis_id = row['id']
        oai_id   = row['oai_identifier_str']
        title    = row.get('title')
        tipo     = row.get('format')
        url      = row.get('url')
        pdate    = row.get('publishDate')
        lang     = row.get('language')
        authors  = row.get('author')
        
        exec_flag = True


        dark_pid = pid_map[oasis_id]['dark']
        pid_python_object = dm.get_pid_by_ark(dark_pid)
        pid_hash = pid_python_object.to_dict()['pid_hash']

        if cmd == 'add_url':
            tx = dark_map.async_set_url(pid_hash,url)
            # request, resp_time, amount = measure_tx_params(dark_gw,call_func,dark_pid,url)
        elif cmd == 'add_external_pid':
            pass
            # if tipo == 'article':
            #     doi = cr_client.get_doi_by_title(title)
            #     if doi != None:
            #         doi = 'doi:/' + doi
            #         request, resp_time, amount = measure_tx_params(dark_gw,call_func,dark_pid,doi)
            #     else:
            #         exec_flag = False
            # else:
            #     exec_flag = False

        elif cmd == 'set_payload':
            payload_data = '{' + f'title : {title}, authors : {authors}, publish_data: {pdate}, oai: {oai_id} , language: {lang}' + '}'
            tx = dark_map.async_set_payload(pid_hash,payload_data)
            # request, resp_time, amount = measure_tx_params(dark_gw,call_func,dark_pid,payload_data)
            

        
        # if exec_flag == True:
        #     # print(request[2])
        #     if request[2].status_code == 200:
        #         status = 'ok'
        #     else:
        #         status = 'error'
        #     #
        # action_map[oasis_id] = {'dark' : dark_pid, 'status' :  status , 'exec_time' : resp_time , 'amount' : amount , 'action' : cmd}
        action_map[oasis_id] = {'dark' : dark_pid, 'status' :  '-' ,  'tx' : tx , 'action' : cmd}
    
        

    return action_map

In [39]:
len(df)
data = df.sample(100)

47820

In [11]:
pid_map = get_pids(data,dm)

In [14]:
urls_txs = call_pid_method('add_url',data,pid_map,dm)

In [15]:
payload_txs = call_pid_method('set_payload',data,pid_map,dm)

## Recuperando um pid aleatorio

In [38]:
import random

index = random.randint(0,len(data))
qdark = pid_map[list(pid_map.keys())[index]]['dark']
dm.get_pid_by_ark(qdark).to_dict()

{'pid_hash': HexBytes('0xb31a660b46f0be0ff10fef91cb24d3f5ecf5843463a88c4ef24a69d0ba601862'),
 'ark': '8033/fkwff300000000003pr',
 'external_pid_list': [],
 'external_url': 'HTTPS://REPOSITORIO.UFPB.BR/JSPUI/HANDLE/123456789/22498',
 'payload': '{title : Farofa social: a organização social do complexo turístico em Barra Grande-PI à luz dos afetos da prática da gourmetização, authors : Olivindo, Celina Maria de Souza, publish_data: 2021, oai: oai:repositorio.ufpb.br:123456789/22498 , language: por}',
 'responsible': '0xf17f52151EbEF6C7334FAD080c5704D77216b732'}

## Old

In [ ]:
a_pid = dm.sync_request_pid()
pid_python_object = dm.get_pid_by_ark(a_pid)
pid_hash = pid_python_object.to_dict()['pid_hash']
pid_python_object.to_dict()

txs_a = []
txs_b = []

print('for')

for i in range(5):
    print(i)
    tx_set2 = dm.async_set_url(pid_hash,'http://www.uol.com.a' + str(i))
    txs_a.append(tx_set2)
    # tx_status, tx_recipt = dgw.transaction_was_executed(tx_set2)

time.sleep(4)

for i in range(5):
    tx_set2 = dm.async_set_url(pid_hash,'http://www.uol.com.b' + str(i))
    txs_b.append(tx_set2)


In [ ]:
dm.get_pid_by_ark(a_pid).to_dict()

In [ ]:
txs_a,txs_b

In [ ]:
for i in range(10):
    print(i)
    tx_set2 = dm.async_set_external_pid(pid_hash,'pid_a' + str(i))
    txs_a.append(tx_set2)
    # tx_status, tx_recipt = dgw.transaction_was_executed(tx_set2)

In [ ]:
dm.get_pid_by_ark(a_pid).to_dict()